 # Skin Diasese detection test

In [12]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [13]:
tf.__version__

'2.10.0'

## Processing the training set

In [14]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    brightness_range=[0.8,1.2],
    fill_mode='nearest'
)
training_set = train_datagen.flow_from_directory(
    'C:/Users/User/Downloads/6_skin_diseases/kaggle/train',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
)

Found 30909 images belonging to 6 classes.


## Preprocessing the test set

In [15]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(
    'C:/Users/User/Downloads/6_skin_diseases/kaggle/test',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
)

Found 3928 images belonging to 6 classes.


## Building the CNN

In [19]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Sequential
# 3. Load Pretrained MobileNetV2 (Exclude top layers)
base_model = MobileNetV2(input_shape=(128, 128, 3),
                         include_top=False,
                         weights='imagenet')
base_model.trainable = False  # Freeze base model

# 4. Add Custom Classification Head
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(6, activation='softmax')  # 6 skin disease classes
])

# 5. Compile Model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 6. Train Model
model.fit(training_set,
          validation_data=test_set,
          epochs=15,
          verbose=1)


9406464/9406464 [==============================] - 2s 0us/step
Epoch 1/15
966/966 [==============================] - 396s 408ms/step - loss: 1.1018 - accuracy: 0.5482 - val_loss: 0.9746 - val_accuracy: 0.5995
Epoch 2/15
966/966 [==============================] - 402s 416ms/step - loss: 0.9917 - accuracy: 0.5978 - val_loss: 0.9120 - val_accuracy: 0.6247
Epoch 3/15
966/966 [==============================] - 397s 411ms/step - loss: 0.9611 - accuracy: 0.6084 - val_loss: 0.9319 - val_accuracy: 0.6255
Epoch 4/15
966/966 [==============================] - 394s 408ms/step - loss: 0.9415 - accuracy: 0.6176 - val_loss: 0.9209 - val_accuracy: 0.6247
Epoch 5/15
966/966 [==============================] - 392s 406ms/step - loss: 0.9266 - accuracy: 0.6262 - val_loss: 0.8989 - val_accuracy: 0.6405
Epoch 6/15
966/966 [==============================] - 390s 404ms/step - loss: 0.9202 - accuracy: 0.6291 - val_loss: 0.9020 - val_accuracy: 0.6309
Epoch 7/15
966/966 [==============================] - 393s 40

In [20]:
model.save('mobilenet_skin_model.h5')

In [16]:
cnn = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same', input_shape=(128, 128, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(6, activation='softmax')
])


In [17]:
cnn.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

In [18]:
cnn.fit(x = training_set, validation_data = test_set, epochs=25, verbose=1)

Epoch 1/25
966/966 [==============================] - 515s 532ms/step - loss: 1.6508 - accuracy: 0.3706 - val_loss: 1.4085 - val_accuracy: 0.4282
Epoch 2/25
 60/966 [>.............................] - ETA: 7:14 - loss: 1.4072 - accuracy: 0.4026

KeyboardInterrupt: 

In [ ]:
cnn.save('6_skin_disease_model.h5')  # or .keras

In [ ]:
import numpy as np
from tensorflow.keras.utils import load_img, img_to_array  # ✅ Modern import

# Load and preprocess the image
test_image = load_img('C:/Users/User/Downloads/6_skin_diseases/kaggle/single_test/ezm.jpg', target_size=(64, 64))
test_image = img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)

# Predict using the CNN model
result = cnn.predict(test_image)

# Define class labels in the same order as training_set.class_indices
class_labels = [
    "Enfeksiyonel",
    "Ekzama",
    "Akne",
    "Pigment",
    "Benign",
    "Malign"
]

# Get the predicted class index
predicted_index = np.argmax(result[0])
prediction = class_labels[predicted_index]

print("Predicted class:", prediction)


1/1 [==============================] - 0s 119ms/step
Predicted class: Benign
